# Analyze composite jobs

In [1]:
import os
import sys
import subprocess
# To add this RDMC into PYTHONPATH in case you haven't do it
sys.path.append(os.path.dirname(os.path.abspath('')))

import cclib
import numpy as np

from rdmc.view import mol_viewer, freq_viewer

from ipywidgets import interact, fixed, IntSlider
from IPython import display

CRITERIA = {
    'gaussian': ['Force Maximum', 'Force RMS', 'Displacement Maximum', 'Displacement RMS'],
}

%load_ext autoreload
%autoreload 2

## 1. Load the file

In [2]:
log = "/Users/xiaorui/C3ddb server/Calcs/dmbene_5/composite/input.log"
######################################
calc_results = cclib.io.ccread(log)
if not hasattr(calc_results, 'metadata'):
    raise RuntimeError('The cclib cannot path the file!')

## 2. Converged?

In [3]:
print(f'Optimization done?: {calc_results.optdone}')
if calc_results.optdone:
    
    print('The optimized XYZ:\n')
    last_converged_idx = [x for x, y in enumerate(calc_results.optstatus) \
                          if y & calc_results.OPT_DONE > 0][-1]
    xyz = cclib.io.ccwrite(calc_results,
                           outputtype='xyz',
                           indices=last_converged_idx,
                           returnstr=True,)
    print(f'{xyz}')

Optimization done?: True
The optimized XYZ:

11
[Geometry 37]
O     -0.9117980000    0.3429320000    0.1976490000
O     -2.5190680000   -0.1977950000   -0.0264670000
O      2.2773120000   -0.5881960000    0.1773010000
C      0.8098680000    1.3093990000   -0.1226030000
C      1.1932070000   -0.1279600000   -0.0732500000
C     -0.1501130000   -0.8122510000   -0.1926590000
H      0.5145470000    1.7515230000   -1.0660700000
H      1.1592640000    1.9824950000    0.6536640000
H     -0.2799370000   -1.6480160000    0.4976370000
H     -0.4017300000   -1.1242260000   -1.2125920000
H     -2.8814760000    0.3675670000    0.6705660000



## 3. Find the geometry that is the closet to convergence

In [4]:
package = calc_results.metadata['package'].lower()
items = CRITERIA[package]

if not calc_results.optdone:
    # Calculate the normalized target differences
    off_targets = (calc_results.geovalues - calc_results.geotargets) / calc_results.geotargets

    # set ones that meet target to 0s 
    off_targets[off_targets <= 0] = 0

    norm2targets = np.linalg.norm(off_targets, axis=1)
    sorted_conformers = np.argsort(norm2targets)

    best_conf = sorted_conformers[0]
    print(f'The best conformer is {best_conf+1}\n')
    print(f'Criteria:\n{"Item":<30}{"Value":<16}{"Threshold":<16}{"Off target":<16}')
    for item, val, target, off_target in zip(items,
                                 calc_results.geovalues[best_conf].tolist(),
                                 calc_results.geotargets.tolist(),
                                 off_targets[best_conf].tolist()):
        print(f'{item:<30}{val:<16}{target:<16}{off_target:<16.2e}')
    print("\n")
    xyz = cclib.io.ccwrite(calc_results,
                           outputtype='xyz',
                           indices=best_conf,
                           returnstr=True,)
    print(f'{xyz}')

In [5]:
viewer = mol_viewer(xyz, model='xyz')
viewer.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## 4. if TS, visualize the vibration

In [6]:
try:
    num_neg_freq = np.sum(calc_results.vibfreqs < 0)
except AttributeError as e:
    raise RuntimeError('!!!!! THIS OUTPUT DOES NOT CONTAIN FREQUENCY INFORMATION !!!!') from e
    
if num_neg_freq != 1:
    raise RuntimeError('!!!!! THIS IS A INVALID TS JOB WITH MULTIPLE IMAGINARY FREQUENCIES !!!!')
else:
    # Get displacement
    print(f'Imaginary freq: {calc_results.vibfreqs[0]} cm-1')
    ts_disp = calc_results.vibdisps[0]
    
    # Compose the xyz with viberation information
    lines = xyz.splitlines()
    vib_xyz_list = lines[0:2]
    for i, line in enumerate(lines[2:]):
        line = line.strip() + f'{"":12}'+ ''.join([f'{item:<12}' for item in ts_disp[i].tolist()])
        vib_xyz_list.append(line)

    vib_xyz = '\n'.join(vib_xyz_list)

Imaginary freq: -799.5938 cm-1


In [7]:
viewer = freq_viewer(vib_xyz, model='xyz',)
viewer.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## 5. Visualize intermediate conformers

In [8]:
num_conf = calc_results.atomcoords.shape[0]
xyzs = [cclib.io.ccwrite(calc_results, outputtype='xyz',
                         indices=i, returnstr=True,)
        for i in range(num_conf)]

def visualize_intermediate_conformer(idx):
    xyz = xyzs[idx]
    return mol_viewer(xyz, model='xyz').show()

interact(visualize_intermediate_conformer, idx=IntSlider(min=1, max=num_conf-1, step=1))

interactive(children=(IntSlider(value=1, description='idx', max=36, min=1), Output()), _dom_classes=('widget-i…

<function __main__.visualize_intermediate_conformer(idx)>

Print an intermediate conformer

In [9]:
print(xyzs[1])

11
[Geometry 2]
O     -0.8879410000    0.2538510000    0.2065680000
O     -2.3124330000   -0.2096860000   -0.2148330000
O      2.2861150000   -0.4120310000   -0.1432660000
C      0.5464620000    1.2472540000    0.0100190000
C      1.1254390000   -0.0742500000   -0.0235840000
C     -0.0874880000   -0.8656090000    0.1497730000
H      0.4305080000    1.8179540000   -0.8411690000
H      0.6770370000    1.8588340000    0.8268220000
H     -0.0563840000   -1.5088600000    1.0149180000
H     -0.3828930000   -1.5207300000   -0.6273820000
H     -2.8606850000    0.4513610000    0.0218110000

